In [ ]:
# U-Net Architecture for Image Segmentation
# Source: Ronneberger, O., Fischer, P., & Brox, T. (2015). U-net: Convolutional networks for biomedical image segmentation.
# In Medical image computing and computer-assisted intervention–MICCAI 2015: 18th international conference, Munich, Germany,
# October 5-9, 2015, proceedings, part III 18 (pp. 234-241). Springer International Publishing.
# Link: https://link.springer.com/chapter/10.1007/978-3-319-24574-4_28

"""
This implementation of the U-Net architecture is designed for image segmentation tasks. U-Net is a convolutional
neural network architecture that consists of a contracting path (encoder) and an expansive path (decoder), which
enable precise localization by capturing both spatial and contextual information. The network's skip connections
allow high-resolution features from the encoder to be concatenated with the decoder layers, enhancing segmentation
accuracy. This model has been adapted with dropout layers and He-normal initialization for improved performance
and stability.
"""

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Activation, concatenate, Conv2DTranspose,MaxPooling2D,Input,Cropping2D,Lambda,Dropout,BatchNormalization
from tensorflow.keras.models import Model

In [ ]:
img_width = 572
img_height = 572
bands = 3

#ENCODER
input = Input((img_width,img_height,bands))
scaled_input = Lambda(lambda x:x/255)(input)
s1 = Conv2D(64,(3,3),padding = 'valid',activation='relu')(scaled_input)
s1 = Conv2D(64,(3,3),padding = 'valid',activation='relu')(s1)
p1 = MaxPooling2D(pool_size=(2,2))(s1)

s2 = Conv2D(128,(3,3),padding = 'valid',activation='relu')(p1)
s2 = Conv2D(128,(3,3),padding = 'valid',activation='relu')(s2)
p2 = MaxPooling2D(pool_size=(2,2))(s2)

s3 = Conv2D(256,(3,3),padding = 'valid',activation='relu')(p2)
s3 = Conv2D(256,(3,3),padding = 'valid',activation='relu')(s3)
p3 = MaxPooling2D(pool_size=(2,2))(s3)

s4 = Conv2D(512,(3,3),padding = 'valid',activation='relu')(p3)
s4 = Conv2D(512,(3,3),padding = 'valid',activation='relu')(s4)
p4 = MaxPooling2D(pool_size=(2,2))(s4)

#BASE COV
s5 = Conv2D(1024,(3,3),padding = 'valid',activation='relu')(p4)
s5 = Conv2D(1024,(3,3),padding = 'valid',activation='relu')(s5)

#DECODER
u1 = Conv2DTranspose(512,(2,2),strides=(2,2), padding ='same')(s5)
s4 = Cropping2D(((0, 8), (0, 8)))(s4)
u1 = concatenate([s4,u1])
s6 = Conv2D(512,(3,3),padding = 'valid',activation='relu')(u1)
s6 = Conv2D(512,(3,3),padding = 'valid',activation='relu')(s6)

u2 = Conv2DTranspose(256,(2,2),strides=(2,2),padding ='same')(s6)
s3 = Cropping2D(((0, 32), (0, 32)))(s3)
u2 = concatenate([s3,u2])
s7 = Conv2D(256,(3,3),padding = 'valid',activation='relu')(u2)
s7 = Conv2D(256,(3,3),padding = 'valid',activation='relu')(s7)

u3 = Conv2DTranspose(128,(2,2),strides=(2,2),padding ='same')(s7)
s2 = Cropping2D(((0, 80), (0, 80)))(s2)
u3 = concatenate([s2,u3])
s8 = Conv2D(128,(3,3),padding = 'valid',activation='relu')(u3)
s8 = Conv2D(128,(3,3),padding = 'valid',activation='relu')(s8)

u4 = Conv2DTranspose(64,(2,2),strides=(2,2),padding ='same')(s8)
s1 = Cropping2D(((0, 176), (0, 176)))(s1)
u4 = concatenate([s1,u4])
s9 = Conv2D(64,(3,3),padding = 'valid',activation='relu')(u4)
s9 = Conv2D(64,(3,3),padding = 'valid',activation='relu')(s9)
output = Conv2D(1,(1,1),padding = 'valid',activation='sigmoid')(s9)

model = Model(inputs =input , outputs = output)

model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

model.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 572, 572, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda_7 (Lambda)         │ (None, 572, 572, 3)    │              0 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_115 (Conv2D)       │ (None, 570, 570, 64)   │          1,792 │ lambda_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_116 (Conv2D)       │ (None, 568, 568, 64)   │         36,928 │ conv2d_115[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_24          │ (None, 284, 284, 64)   │              0 │ conv2d_116[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_117 (Conv2D)       │ (None, 282, 282, 128)  │         73,856 │ max_pooling2d_24[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_118 (Conv2D)       │ (None, 280, 280, 128)  │        147,584 │ conv2d_117[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_25          │ (None, 140, 140, 128)  │              0 │ conv2d_118[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_119 (Conv2D)       │ (None, 138, 138, 256)  │        295,168 │ max_pooling2d_25[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_120 (Conv2D)       │ (None, 136, 136, 256)  │        590,080 │ conv2d_119[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_26          │ (None, 68, 68, 256)    │              0 │ conv2d_120[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_121 (Conv2D)       │ (None, 66, 66, 512)    │      1,180,160 │ max_pooling2d_26[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_122 (Conv2D)       │ (None, 64, 64, 512)    │      2,359,808 │ conv2d_121[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_27          │ (None, 32, 32, 512)    │              0 │ conv2d_122[0][0]       │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_123 (Conv2D)       │ (None, 30, 30, 1024)   │      4,719,616 │ max_pooling2d_27[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_124 (Conv2D)       │ (None, 28, 28, 1024)   │      9,438,208 │ conv2d_123[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cropping2d_24             │ (None, 56, 56, 512)    │              0 │ conv2d_122[0][0]       │
│ (Cropping2D)         

 Total params: 31,031,745 (118.38 MB)

 Trainable params: 31,031,745 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Enhanced U-Net Architecture with Batch Normalization for Image Segmentation

"""
This implementation upgrades the standard U-Net model by incorporating Batch Normalization layers, Droupout layers, Kernel initializer and padding.
"""

'\nThis implementation upgrades the standard U-Net model by incorporating Batch Normalization layers, Droupout layers, Kernel initializer and padding. \n'

In [3]:
img_width = 572
img_height = 572
bands = 3

# ENCODER
input = Input((img_width, img_height, bands))
scaled_input = Lambda(lambda x: x / 255)(input)

# Encoder block 1
s1 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(scaled_input)
s1 = BatchNormalization()(s1)
s1 = Activation('relu')(s1)
s1 = Dropout(0.1)(s1)
s1 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(s1)
s1 = BatchNormalization()(s1)
s1 = Activation('relu')(s1)
p1 = MaxPooling2D(pool_size=(2, 2))(s1)

# Encoder block 2
s2 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
s2 = BatchNormalization()(s2)
s2 = Activation('relu')(s2)
s2 = Dropout(0.1)(s2)
s2 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(s2)
s2 = BatchNormalization()(s2)
s2 = Activation('relu')(s2)
p2 = MaxPooling2D(pool_size=(2, 2))(s2)

# Encoder block 3
s3 = Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
s3 = BatchNormalization()(s3)
s3 = Activation('relu')(s3)
s3 = Dropout(0.2)(s3)
s3 = Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(s3)
s3 = BatchNormalization()(s3)
s3 = Activation('relu')(s3)
p3 = MaxPooling2D(pool_size=(2, 2))(s3)

# Encoder block 4
s4 = Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
s4 = BatchNormalization()(s4)
s4 = Activation('relu')(s4)
s4 = Dropout(0.2)(s4)
s4 = Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(s4)
s4 = BatchNormalization()(s4)
s4 = Activation('relu')(s4)
p4 = MaxPooling2D(pool_size=(2, 2))(s4)

# Base block
s5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
s5 = BatchNormalization()(s5)
s5 = Activation('relu')(s5)
s5 = Dropout(0.3)(s5)
s5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(s5)
s5 = BatchNormalization()(s5)
s5 = Activation('relu')(s5)

# DECODER
# Decoder block 1
u1 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(s5)
s4 = Cropping2D(((0, 1), (0, 1)))(s4)
u1 = concatenate([s4, u1])
s6 = Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u1)
s6 = BatchNormalization()(s6)
s6 = Activation('relu')(s6)
s6 = Dropout(0.2)(s6)
s6 = Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(s6)
s6 = BatchNormalization()(s6)
s6 = Activation('relu')(s6)

# Decoder block 2
u2 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(s6)
s3 = Cropping2D(((0, 3), (0, 3)))(s3)
u2 = concatenate([s3, u2])
s7 = Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u2)
s7 = BatchNormalization()(s7)
s7 = Activation('relu')(s7)
s7 = Dropout(0.2)(s7)
s7 = Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(s7)
s7 = BatchNormalization()(s7)
s7 = Activation('relu')(s7)

# Decoder block 3
u3 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(s7)
s2 = Cropping2D(((0, 6), (0, 6)))(s2)
u3 = concatenate([s2, u3])
s8 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u3)
s8 = BatchNormalization()(s8)
s8 = Activation('relu')(s8)
s8 = Dropout(0.1)(s8)
s8 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(s8)
s8 = BatchNormalization()(s8)
s8 = Activation('relu')(s8)

# Decoder block 4
u4 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(s8)
s1 = Cropping2D(((0, 12), (0, 12)))(s1)
u4 = concatenate([s1, u4])
s9 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u4)
s9 = BatchNormalization()(s9)
s9 = Activation('relu')(s9)
s9 = Dropout(0.1)(s9)
s9 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(s9)
s9 = BatchNormalization()(s9)
s9 = Activation('relu')(s9)

output = Conv2D(1, (1, 1), activation='sigmoid')(s9)

model = Model(inputs=input, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 572, 572, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 572, 572, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 572, 572, 64)   │          1,792 │ lambda[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 572, 572, 64)   │            256 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 572, 572, 64)   │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 572, 572, 64)   │              0 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 572, 572, 64)   │         36,928 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 572, 572, 64)   │            256 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 572, 572, 64)   │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 286, 286, 64)   │              0 │ activation_1[0][0]     │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 286, 286, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 286, 286, 128)  │            512 │ conv2d_2[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 286, 286, 128)  │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 286, 286, 128)  │              0 │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 286, 286, 128)  │        147,584 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 286, 286, 128)  │            512 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 286, 286, 128)  │              0 │ batch_normalization_3… │
├──────────────────────

 Total params: 31,055,297 (118.47 MB)

 Trainable params: 31,043,521 (118.42 MB)

 Non-trainable params: 11,776 (46.00 KB)